**mounting google drive**

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Getting required libs**

In [11]:
import csv
import pandas as pd
import numpy as np

**Dataset address**

In [12]:
InputFileAddress = '/content/drive/MyDrive/Colab Notebooks/Data management Project/Data Management Project - Datafiles/Part 01.2 input - Dataset after Converting into CSV with correct structure/Part 02 input.csv'
OutputFileAddress = '/content/drive/MyDrive/Colab Notebooks/Data management Project/Data Management Project - Datafiles/Part 01.3 input - Dataset after Preprocessing/PreprocessedDataset.csv'

**reading the csv**

In [13]:
# Read the CSV file using pandas
data = pd.read_csv(InputFileAddress)
# number_of_training_cases = len(data.index)
print(len(data.index))

<ipython-input-13-dfca34e3f173>:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(InputFileAddress)


515738


In [14]:
display(data.head(1))

,Hotel_Address_values,Additional_Number_of_Scoring_values,Review_Date_values,Average_Score_values,Hotel_Name_values,Reviewer_Nationality_values,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,tag_column,days_since_review,latitude,longitude
0,' s Gravesandestraat 55 Oost 1092 AA Amsterdam...,194,8/3/2017,7.7,'Hotel Arena',' Russia ',' I am so angry that i made this post availabl...,397,1403,' Only the park outside of the hotel was beaut...,11,7,2.9,"['2.9', ""'[\\' Leisure trip \\'"", "" \\' Couple...",'0 days',52.360576,4.915968


**Handling Missing Addresses**

In [15]:
# Find and replace missing addresses by matching hotel name, latitude, and longitude
data['Hotel_Address_values'] = data.groupby(['Hotel_Name_values', 'latitude', 'longitude'])['Hotel_Address_values'].transform(lambda x: x.fillna(x.mode().iloc[0]))
display(data.head(1))

,Hotel_Address_values,Additional_Number_of_Scoring_values,Review_Date_values,Average_Score_values,Hotel_Name_values,Reviewer_Nationality_values,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,tag_column,days_since_review,latitude,longitude
0,' s Gravesandestraat 55 Oost 1092 AA Amsterdam...,194,8/3/2017,7.7,'Hotel Arena',' Russia ',' I am so angry that i made this post availabl...,397,1403,' Only the park outside of the hotel was beaut...,11,7,2.9,"['2.9', ""'[\\' Leisure trip \\'"", "" \\' Couple...",'0 days',52.360576,4.915968


**Formatting Review Date**

In [16]:
# Validate and format the Review_Date to 'YYYY-MM-DD' format
data['Review_Date_values'] = pd.to_datetime(data['Review_Date_values'], errors='coerce')
data['Review_Date_values'] = data['Review_Date_values'].dt.strftime('%Y-%m-%d')
display(data.head(1))

,Hotel_Address_values,Additional_Number_of_Scoring_values,Review_Date_values,Average_Score_values,Hotel_Name_values,Reviewer_Nationality_values,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,tag_column,days_since_review,latitude,longitude
0,' s Gravesandestraat 55 Oost 1092 AA Amsterdam...,194,2017-08-03,7.7,'Hotel Arena',' Russia ',' I am so angry that i made this post availabl...,397,1403,' Only the park outside of the hotel was beaut...,11,7,2.9,"['2.9', ""'[\\' Leisure trip \\'"", "" \\' Couple...",'0 days',52.360576,4.915968


**Filling Missing Average Scores**

In [17]:
# Convert 'Review_Date' column to datetime
data['Review_Date_values'] = pd.to_datetime(data['Review_Date_values'], errors='coerce')


# Fill missing Average_Score values based on similar reviews in the same year
data['Average_Score_values'] = data.groupby(['Hotel_Name_values', 'latitude', 'longitude', data['Review_Date_values'].dt.year])['Average_Score_values'].transform(lambda x: x.fillna(x.mean()))
display(data.head(1))

,Hotel_Address_values,Additional_Number_of_Scoring_values,Review_Date_values,Average_Score_values,Hotel_Name_values,Reviewer_Nationality_values,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,tag_column,days_since_review,latitude,longitude
0,' s Gravesandestraat 55 Oost 1092 AA Amsterdam...,194,2017-08-03,7.7,'Hotel Arena',' Russia ',' I am so angry that i made this post availabl...,397,1403,' Only the park outside of the hotel was beaut...,11,7,2.9,"['2.9', ""'[\\' Leisure trip \\'"", "" \\' Couple...",'0 days',52.360576,4.915968


**Cleaning Negative and Positive Reviews**

In [18]:
import re

# Function to clean and preprocess text data
def clean_text(text):
    # Remove non-alphanumeric characters and extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ' '.join(text.split())
    return text

# Apply the clean_text function to Negative_Review and Positive_Review columns
data['Negative_Review'] = data['Negative_Review'].apply(clean_text)
data['Positive_Review'] = data['Positive_Review'].apply(clean_text)

**Saving data**

In [19]:
# Define the file path in Google Drive
file_path = OutputFileAddress

# Save the DataFrame to a CSV file
data.to_csv(file_path, index=False)

# Confirm the file was saved
print(f"File saved to {file_path}")

File saved to /content/drive/MyDrive/Colab Notebooks/Data management Project/Data Management Project - Datafiles/Part 01.3 input - Dataset after Preprocessing/PreprocessedDataset.csv
